In [1]:
%load_ext lab_black

In [2]:
from pyprojroot import here
import os
from skimpy import clean_columns
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import shap
import joblib
from joblib import load

path_data = here("./data")
os.chdir(path_data)

data = pd.read_csv("data_pmpm.csv")
data_predict = data.drop(columns=["Hospital ID", "Per Member Per Month Cost"])
# Load the model
model = joblib.load("model_drivers.joblib")

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Get the actual percentage of members with ER visits in the last 30 days for each hospital

Show them their average percentages for all four variables.

Add some noise for the features by Hospital ID

Then create a function to allow for simulated data with the exact percentage that someone shows

In [4]:
data.groupby("Hospital ID").mean().reset_index().round(2)

,Hospital ID,High Blood Pressure,High Cholesterol,Diabetes,Preventative Services,Per Member Per Month Cost
0,1,0.3,0.40,0.25,0.5,5178.47
1,2,0.3,0.40,0.25,0.5,5163.05
2,3,0.3,0.41,0.25,0.5,5179.44


Step: Create a function that generates 

In [5]:
import pandas as pd
import numpy as np


def adjust_binary_percentages(df, **column_percentages):
    """
    Adjusts the percentage of 1's in each specified column of a binary DataFrame.

    Args:
    df (pd.DataFrame): DataFrame with binary values.
    column_percentages (dict): A dictionary where keys are column names and values are the new desired percentages of 1's.

    Returns:
    pd.DataFrame: Modified DataFrame.
    """

    for column, percentage in column_percentages.items():
        if column not in df.columns:
            raise ValueError(f"Column {column} not found in DataFrame")

        # Calculate current percentage of 1's
        current_percentage = df[column].mean()

        # Calculate the desired number of 1's
        target_count = int(df.shape[0] * percentage)

        # Find indices where changes are needed
        ones_indices = df[df[column] == 1].index
        zeros_indices = df[df[column] == 0].index

        if target_count > ones_indices.size:  # Need to add more 1's
            change_count = target_count - ones_indices.size
            indices_to_change = np.random.choice(
                zeros_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 1
        else:  # Need to remove some 1's
            change_count = ones_indices.size - target_count
            indices_to_change = np.random.choice(
                ones_indices, change_count, replace=False
            )
            df.loc[indices_to_change, column] = 0

    return df

In [6]:
data_predict_adjust = clean_columns(data_predict.copy())
data_predict_adjust = adjust_binary_percentages(
    df=data_predict_adjust,
    high_blood_pressure=0.1,
    high_cholesterol=0.1,
    diabetes=0.5,
    preventative_services=0.7,
)
data_predict_adjust.rename(
    columns={
        "high_blood_pressure": "High Blood Pressure",
        "high_cholesterol": "High Cholesterol",
        "diabetes": "Diabetes",
        "preventative_services": "Preventative Services",
    },
    inplace=True,
)

In [7]:
# Make predictions (opxtional, to evaluate model)
predictions = model.predict(data_predict_adjust)
predictions_pd = pd.DataFrame(predictions).rename(columns={0: "Predictions"})

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [8]:
data_predictions_hospital_id = pd.concat([data["Hospital ID"], predictions_pd], axis=1)

data_predictions_hospital_group = (
    data_predictions_hospital_id.groupby("Hospital ID").mean().reset_index().round(2)
)

noise = np.random.uniform(
    -100, 100, data_predictions_hospital_group["Predictions"].shape
)
data_predictions_hospital_group["Predictions"] = (
    data_predictions_hospital_group["Predictions"]  # + noise
).round(2)

data_predictions_hospital_group

,Hospital ID,Predictions
0,1,4121.000000
1,2,4136.209961
2,3,4143.220215


In [14]:
data_actual_costs = (
    data[["Hospital ID", "Per Member Per Month Cost"]]
    .groupby("Hospital ID")
    .mean()
    .reset_index()
).round(0)
data_actual_costs

,Hospital ID,Per Member Per Month Cost
0,1,5178.0
1,2,5163.0
2,3,5179.0


In [35]:
import pandas as pd

# Assuming data_actual_costs and data_predictions_hospital_group are already defined

# Merge the datasets on 'Hospital ID'
merged_data = data_actual_costs.merge(
    data_predictions_hospital_group,
    on="Hospital ID",
)

# Calculate the difference
merged_data["Difference"] = (
    merged_data["Predictions"] - merged_data["Per Member Per Month Cost"]
)

# Calculate the percentage change
merged_data["Percentage Change"] = (
    merged_data["Difference"] / merged_data["Predictions"]
) * 100

# Formatting the Difference and Percentage Change
merged_data["Difference"] = merged_data["Difference"].map("${:,.0f}".format)
merged_data["Predictions"] = merged_data["Predictions"].map("${:,.0f}".format)
merged_data["Per Member Per Month Cost"] = merged_data["Per Member Per Month Cost"].map(
    "${:,.0f}".format
)


merged_data["Percentage Change"] = merged_data["Percentage Change"].map(
    "{:.2f}%".format
)

# Concatenating the relevant columns
data_predicted_actual = pd.concat(
    [
        merged_data[["Hospital ID"]],
        merged_data[["Predictions"]],
        merged_data[["Per Member Per Month Cost", "Difference", "Percentage Change"]],
    ],
    axis=1,
)

# Display the final DataFrame
data_predicted_actual

,Hospital ID,Predictions,Per Member Per Month Cost,Difference,Percentage Change
0,1,"$4,121","$5,178","$-1,057",-25.65%
1,2,"$4,136","$5,163","$-1,027",-24.82%
2,3,"$4,143","$5,179","$-1,036",-25.00%


In [15]:

        # Calculate difference and percentage change



KeyError: 'Per Member Per Month Cost_Actual'